In [24]:
import pandas as pd
df = pd.read_csv('filter0.csv')

In [25]:
df = df[['등록번호','회복실퇴실일시','최대 NRS','KEY']]

# 2022-01-21 12:13:00 > 210121, 1213

df = df.reset_index(drop=True)
def get_date(date:str,type_='ymd'):
    
    date = date.split()

    y,m,d = date[0].split('-')
    h,min_   = date[1].split(':')
    
    out = ''
    if type_ == 'ymd':
        ymd = y[-2:]+m+d
        out = ymd
    elif type_ == 'hm':
        hm = h+min_
        out = hm
    return out

df['time'] = df['회복실퇴실일시'].map(lambda x: get_date(x,'hm') )

df['date'] = df['회복실퇴실일시'].map(lambda x: get_date(x,'ymd') )

In [26]:
# change the cols. sequence 
df = df[['등록번호','date','time','최대 NRS','KEY']]

In [28]:
# prepare the materials
import pandas as pd 
df = pd.read_csv('filter0_simple_1.csv')

In [30]:
import pickle 
with open('vital_paths.pkl','rb') as f:
    allf = pickle.load(f)
with open('vital_fname.pkl','rb') as f:
    allf_n = pickle.load(f)
with open('vital_split.pkl','rb') as f:
    allf_s = pickle.load(f)

In [32]:
df.columns = ['pt_id', 'date', 'time', 'max_nrs','key']

df.max_nrs = df.max_nrs.map(round)

In [33]:
# (vital-existing only)
idx = []
for j, sample in enumerate(df[['pt_id','date','time','max_nrs','key']].values): # sample: ['등록번호','date','time','최대 NRS']
    
    sample = sample.astype('str')
                           
    for i, data in enumerate(allf_s): # data: ['00000824', 'ORC14', '210611', '133500', '1']

        if data[0].endswith(sample[0]): # 등록번호
            if data[2].startswith(sample[1]): # 회복실퇴실일시(date) 210611
                idx.append([j, sample,allf_n[i],allf[i],sample[-2],sample[-1] ])

dfi = pd.DataFrame(idx,columns = ['idx','label','vital','path','nrs','key'])


In [35]:
import os
def extract_path(path,paths=['e:/2021_1_1_10_14/충남대병원/마취통증의학과/OTHER','e:/2021_1015_1231/충남대병원/마취통증의학과/OTHER','e:\\2022\\충남대병원\\마취통증의학과\\OTHER']):
    ext_path = ''
    
    for p in paths:
        if os.path.commonpath([path,p]) == os.path.abspath(p):
            ext_path = os.path.relpath(path,start=p)
            break
    return ext_path
    
ext_path = dfi.path.map(extract_path)

import pandas as pd
dfi['ext_path'] = ext_path

In [36]:
# pre&post pair 
# idx duplicated (con1. pre&post pair)
dfi = dfi[dfi.idx.duplicated(keep=False)]

# pre&post exists? (con2. pre&post pair)
import re
pa = re.compile('\D+')

indices = []
for i in dfi.idx.unique():
    duplicates = dfi[dfi.idx == i]
    flist = [ s.split('_')[1].lower() for s in duplicates['vital'].to_list()] # ['orc8', 'orc8', 'orc8']
    flist = [pa.findall(f)[0] for f in flist]
    
    if ('pdor' in flist) and ('pdrec' in flist):
        indices.append(i)

dfip = dfi[dfi.idx.isin(indices)]

In [49]:
def ext_room(name,num=False):
    room_n = name.split('_')[1]
    if num:
        return room_n
    else:
        room = pa.findall(room_n)[0].lower()
        return room
dfip['room'] = dfip.vital.map(ext_room)

C:\Users\USER\anaconda3\envs\p37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
dfip.to_csv('./dfi_4key.csv',index=False)

### json 

In [63]:
# path_0
path_0 =  []

for i in range(len(dfip)):
    name = dfip['vital'].iloc[i]
    folder_n = dfip['room'].iloc[i]
    path_0.append(os.path.join(".\\"+folder_n, name+'.vital') )
    
dfip['path_0'] = path_0

C:\Users\USER\anaconda3\envs\p37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
# 'key', 'room', 'path_0', 'nrs'

In [91]:
samples = []
for idx in dfip.idx.unique():
    
    df_idx = dfip[dfip.idx == idx]
    
    # df_idx['path_0']
    or_paths = []
    rec_paths = []
    for room, p in df_idx[['room','path_0']].values:
        if room == 'pdor':
            or_paths.append(p)
        elif room == 'pdrec':
            rec_paths.append(p)
        else:
            raise ValueError('Noe-existing room')
    
    sample = {}
    sample['key'] = df_idx['key'].iloc[0]
    sample['rec_path'] = rec_paths
    sample['or_path'] = or_paths
    sample['nrs'] = df_idx['nrs'].iloc[0]
    samples.append(sample)

In [117]:
with open('all.json', 'w') as f:
    json.dump(samples, f)

### add columns 

In [ ]:
def add_cols(jpath, fpath, cols):
    jpath = '../data/pd_gy/all.json'
    fpath = 'filter0.csv'

    add_cols = ['age','gender']

    import json 
    with open(jpath,'r') as f:
        dj = json.load(f)

    import pandas as pd
    op = pd.read_csv(fpath)

    op = op.rename(columns = {'성별':'gender','나이':'age','KEY':'key'})
    op['key'] = op.key.astype('str')

    l = []
    for i,val in enumerate(dj):
        key  = val['key']
        item = op[op.key==key]
        for col in add_cols:
            col_value = item[col].values[0]
            dj[i][col] = col_value
    return dj

In [ ]:
jpath = '../data/pd_gy/all.json'
fpath = 'filter0.csv'

import json 
with open(jpath,'r') as f:
    dj = json.load(f)

import pandas as pd
op = pd.read_csv(fpath)

op = op.rename(columns = {'성별':'gender','나이':'age','KEY':'key'})
op['key'] = op.key.astype('str')

l = []
for i,val in enumerate(dj):
    key  = val['key']
    item = op[op.key==key]
    age = item['age'].values[0] 
    gender = item['gender'].values[0]
    dj[i]['age'], dj[i]['gender'] = age, gender.lower()